In [1]:
!pip install pytesseract

In [2]:
!pip install opencv-python

In [3]:
!pip install pymupdf

In [61]:
import os
import cv2
import fitz
import itertools
import pytesseract
import numpy as np
import pandas as pd
from PIL import Image

In [62]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [63]:
os.environ['PATH'] += r';C:\Program Files\Tesseract-OCR'

Обобщим обработку на случай pdf файла на входе, сделаем пердобработку и выполним нарезку

In [273]:
class TableExtraction:
    def __init__(self):
        self.input_data = []
        self.list_of_np_arrays = []
        self.threshold_images = []
        self.parts_boxes = []
        self.parts_image = []

    def convert_file_to_array(self, file_path):
        file_extension = os.path.splitext(file_path)[1]
        if file_extension == '.pdf':
            doc = fitz.open(file_path)
            for n in range(doc.page_count):
                page = doc.load_page(n)
                pix = page.get_pixmap()
                image = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
                image = np.ascontiguousarray(image[..., [2, 1, 0]])
                self.list_of_np_arrays.append(image)
        else:
            raise ValueError('Unsupported file format')

    def binarization(self):
        for image in self.list_of_np_arrays:
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Применение порогового преобразования для бинаризации изображения
            _, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY)
            self.threshold_images.append(threshold_image)

    def split_pages(self):
        for num, image in enumerate(self.list_of_np_arrays):
            # Преобразование изображения в оттенки серого
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Применение порогового преобразования для бинаризации изображения
            ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

            # Применение морфологического преобразования для удаления шума и замыкания линий
            kernel = np.ones((5, 5), np.uint8)
            morph_image = cv2.morphologyEx(threshold_image, cv2.MORPH_CLOSE, kernel)

            # Нахождение контуров на изображении
            contours, hierarchy = cv2.findContours(morph_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            height, width, _ = image.shape
            # Фильтрация контуров и определение ограничивающих рамок ячеек
            for contour in contours:
                # Определение площади контура
                area = cv2.contourArea(contour)

                # Фильтрация контуров по площади
                if area > 3000:
                    # Определение ограничивающей рамки контура
                    x, y, w, h = cv2.boundingRect(contour)

                    # Добавление ограничивающей рамки в список
                    self.parts_boxes.append((x, y, x + w, y + h))
                    x1, y1, x2, y2 = self.parts_boxes[-1]
                    self.parts_image.append(image[y1:y2, x1:x2])

            # Вывод координат ограничивающих рамок ячеек
            for box in self.parts_boxes:
                x1, y1, x2, y2 = box
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    def extract(self):
        for num, image in enumerate(self.parts_image):

            # Преобразование изображения в оттенки серого
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Получение ограничивающих рамок символов
            custom_config = r'--oem 1 --psm 1 -l rus'
            boxes = pytesseract.image_to_boxes(gray_image, config=custom_config)

            # Разделение строки на список строк
            box_lines = boxes.strip().split('\n')

            # Преобразование каждой строки в список
            boxes_list = [line.split() for line in box_lines]

            # Так как cv2 и tesseract используют разные системы координат
            height, width, _ = image.shape

            # Список границ таблицы
            borders = []

            # Вывод ограничивающих рамок символов
            if boxes_list[0] == []:
                continue
            for box in boxes_list:
                if box[0] == '~':
                    x, y, w, h = int(box[1]), int(box[2]), int(box[3]), int(box[4])
                    borders.append([x, y, w, h])

            if len(borders) > 1:
                # Находим все вертикальные и горизонтальные линии каркаса таблицы
                vertical_lines = []
                horizontal_lines = []

                # Параметры допуска при котором считаем что линии ровные и пересекаются
                epsilon = 5

                for x1, y1, x2, y2 in borders:
                    if abs(x1 - x2) < epsilon:  # Вертикальная линия
                        vertical_lines.append((x1, y1, x2, y2))
                    elif abs(y1 - y2) < epsilon:  # Горизонтальная линия
                        horizontal_lines.append((x1, y1, x2, y2))

                # Находим координаты узлов таблицы
                table_nodes = []
                for v_line, h_line in itertools.product(vertical_lines, horizontal_lines):
                    x, y = v_line[0], h_line[1]
                    cv2.rectangle(image, (x, height - y), (x + 3, height - y + 3), (0, 0, 255), 1)
                    table_nodes.append((x, y))

                if table_nodes:
                    table_nodes = np.array(table_nodes)

                    # Замена координат, отличающихся менее чем на epsilon
                    rounded_nodes = np.round(table_nodes / epsilon) * epsilon

                    # Подсчет уникальных значений координат x и y
                    unique_x = np.unique(rounded_nodes[:, 0])
                    unique_y = np.unique(rounded_nodes[:, 1])

                    # Определение числа строк и столбцов таблицы
                    num_rows = len(unique_y) - 1
                    num_columns = len(unique_x) - 1

                    # Отсортируем узлы сначала по оси x, потом по оси y
                    nodes_sorted_x = sorted(table_nodes, key=lambda x: (x[0], x[1]))

                    # Создадим список прямоугольников
                    rectangles = []

                    # Пройдем по узлам и создадим прямоугольники
                    for i in range(len(nodes_sorted_x) - 1):
                        current_node = nodes_sorted_x[i]
                        if nodes_sorted_x[i + 1][0] == current_node[0]:
                            next_y_node = nodes_sorted_x[i + 1]
                        else:
                            continue
                        # Найдем узлы следующие по оси y для текущих узлов
                        next_x_nodes = [node for node in nodes_sorted_x if node[1] == current_node[1] and node[0] > current_node[0]]
                        if next_x_nodes:
                            next_x_node = min(next_x_nodes, key=lambda x: x[0])
                            opposite_node = (next_x_node[0], next_y_node[1])
                            # Построим прямоугольник по найденным узлам
                            rectangle = ((current_node[0], current_node[1]), (opposite_node[0], opposite_node[1]))
                            rectangles.append(rectangle)
                        else:
                            continue

                    rectangles = sorted(rectangles, key=lambda x: (- x[0][1], x[0][0]))
                    # Создание пустого массива для хранения данных таблицы
                    table_data = [['' for _ in range(num_columns)] for _ in range(num_rows)]

                    # Обход прямоугольников ячеек таблицы
                    for i, rectangle in enumerate(rectangles):
                        # Извлечение координат углов прямоугольника
                        x1, y1 = rectangle[0]
                        x2, y2 = rectangle[1]

                        # Обрезание изображения по границам ячейки
                        cell_image = image[max(0,(height - y2) - 2):min(height,(height - y1) + 2), max(0, x1 - 2):min(width, x2 + 2)]

                        if cell_image.shape[0] > 10 and cell_image.shape[1] > 10:

                            # Распознавание текста с помощью Tesseract
                            custom_config = r'--oem 1 --psm 1 -l rus'
                            text = pytesseract.image_to_string(cell_image, config=custom_config).replace('\n', ' ')

                            # Определение положения ячейки в таблице
                            row = i // num_columns
                            column = i % num_columns

                            # Добавление данных ячейки
                            if row < num_rows and column < num_columns:
                                table_data[row][column] = text

                    # Вывод полученной таблицы
                    df = pd.DataFrame(table_data)
                    print(table_data)
                    # Сохранение DataFrame в формате CSV
                    df.to_csv(f'result/table_{num}.csv', index=False, header=False)

In [275]:
extractor = TableExtraction()
extractor.convert_file_to_array('test.pdf')
extractor.binarization()
extractor.split_pages()
extractor.extract()

[["'Облучаемая труппа населения  ", 'Коэффициент риска злокачественных новообразований,  х10-2 Зв-1  ', 'Коэффициент риска ‘наследственных  эффектов, х10-2 Зв-1  ', '‘Сумма, х10-2 Зв-1  '], ['Все население  ', '5,5  ', '0,2  ', '5,7  '], ['Взрослые  ', '4,1  ', '0,1  ', '4,2  ']]
[['', '', ''], ['', '', ''], ['', 'о сдлаоума ВС ии, а  ', ''], ['', '', '']]
[['', '', '', '', '', '', ''], ['', 'Зарегистрированные преступления.  ', '2015  ', '2016  ', '2017  ', '2015  ', ''], ['', '', '', '', '288  19  ', '500  00  ', ''], ['', '', '', '', '', '1,00.  ', ''], ['', 'Разбойные нападения  ', '', '', '', '', ''], ['', 'Мошенничество (ст.159 — ст. 159 пр.6)  ', '56.  ', '67.  ', '79.  ', '11.00  ', ''], ['', '', '', '', '1.00  ', '0.00  ', ''], ['', 'Умыш. уничтожение или поврежден. чужого имущества.  ', '12  ', '14  ', '14  ', '', '']]
[['', '', 'овка регионов)  ', '', ''], ['', 'Таблица 2 бон} ада промышленности  пыбросоь в атиостферу  ', '', '', ''], ['', '', '', '', ''], ['', '', '', '', '

In [228]:
def convert_file_to_array(file_path):
    file_extension = os.path.splitext(file_path)[1]
    if file_extension == '.pdf':
        doc = fitz.open(file_path)
        list_of_np_arrays = []
        for n in range(doc.page_count):
            page = doc.load_page(n)
            pix = page.get_pixmap()
            image = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
            image = np.ascontiguousarray(image[..., [2, 1, 0]])
            list_of_np_arrays.append(image)
        return list_of_np_arrays
    else:
        raise ValueError('Unsupported file format')


# Пример использования
file_path = 'table.pdf'
file_array = convert_file_to_array(file_path)

print(file_array[0].shape)

(294, 303, 3)


In [70]:
# Загрузка изображения
image = cv2.imread('draft/image.png')

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Получение ограничивающих рамок символов
custom_config = r'--oem 1 --psm 1 -l rus'
boxes = pytesseract.image_to_boxes(gray_image, config=custom_config)

# Разделение строки на список строк
box_lines = boxes.strip().split('\n')

# Преобразование каждой строки в список
boxes_list = [line.split() for line in box_lines]

# Так как cv2 и tesseract используют разные системы координат
height, width, _ = image.shape

# Список границ таблицы
borders = []

# Вывод ограничивающих рамок символов
for box in boxes_list:
    if box[0] == '~':
        x, y, w, h = int(box[1]), int(box[2]), int(box[3]), int(box[4])
        borders.append([x, y, w, h])
        cv2.rectangle(image, (x, height - y), (w, height - h), (0, 255, 255), 1)

# # Отображение изображения с ограничивающими рамками символов
# cv2.imshow('Image with Character Boxes', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [71]:
# Находим все вертикальные и горизонтальные линии каркаса таблицы
vertical_lines = []
horizontal_lines = []

# Параметры допуска при котором считаем что линии ровные и пересекаются
epsilon = 3

for x1, y1, x2, y2 in borders:
    if abs(x1 - x2) < epsilon:  # Вертикальная линия
        vertical_lines.append((x1, y1, x2, y2))
    elif abs(y1 - y2) < epsilon:  # Горизонтальная линия
        horizontal_lines.append((x1, y1, x2, y2))

# Находим координаты узлов таблицы
table_nodes = []
for v_line, h_line in itertools.product(vertical_lines, horizontal_lines):
    x, y = v_line[0], h_line[1]
    # cv2.rectangle(image, (x, height - y), (x + 3, height - y + 3), (0, 0, 255), 1)
    table_nodes.append((x, y))

# # Отображение изображения
# cv2.imshow('Image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [324]:
# Параметры epsilon и table_nodes
epsilon = 5
table_nodes = np.array(table_nodes)

# Замена координат, отличающихся менее чем на epsilon, на среднее значение
rounded_nodes = np.round(table_nodes / epsilon) * epsilon

# Подсчет уникальных значений координат x и y
unique_x = np.unique(rounded_nodes[:, 0])
unique_y = np.unique(rounded_nodes[:, 1])

# Определение числа строк и столбцов таблицы
num_rows = len(unique_y) - 1
num_columns = len(unique_x) - 1

# Отсортируем узлы по оси x
nodes_sorted_x = sorted(table_nodes, key=lambda x: (x[0], x[1]))

# Создадим список прямоугольников
rectangles = []

# Пройдем по узлам и создадим прямоугольники
for i in range(len(nodes_sorted_x) - 1):
    current_node = nodes_sorted_x[i]
    if nodes_sorted_x[i + 1][0] == current_node[0]:
        next_y_node = nodes_sorted_x[i + 1]
    else:
        continue
    # Найдем узлы следующие по оси y для текущих узлов
    next_x_nodes = [node for node in nodes_sorted_x if node[1] == current_node[1] and node[0] > current_node[0]]
    if next_x_nodes:
        next_x_node = min(next_x_nodes, key=lambda x: x[0])
        opposite_node = (next_x_node[0], next_y_node[1])
        # Построим прямоугольник по найденным узлам
        rectangle = ((current_node[0], current_node[1]), (opposite_node[0], opposite_node[1]))
        rectangles.append(rectangle)
        # cv2.rectangle(image, (current_node[0] + 2, height - current_node[1] + 2), (opposite_node[0] - 2, height - opposite_node[1] - 2), (255, 255, 0), 1)
    else:
        continue

rectangles = sorted(rectangles, key=lambda x: (- x[0][1], x[0][0]))
# Создание пустого массива для хранения данных таблицы
table_data = [['' for _ in range(num_columns)] for _ in range(num_rows)]

# Обход прямоугольников ячеек таблицы
for i, rectangle in enumerate(rectangles):
    # Извлечение координат углов прямоугольника
    x1, y1 = rectangle[0]
    x2, y2 = rectangle[1]

    # Обрезание изображения по границам ячейки
    cell_image = image[(height - y2):(height - y1), x1:x2]

    # Распознавание текста с помощью Tesseract
    custom_config = r'--oem 1 --psm 1 -l rus'
    text = pytesseract.image_to_string(cell_image, config=custom_config).replace('\n', ' ')

    # Определение положения ячейки в таблице
    row = i // num_columns
    column = i % num_columns

    # Добавление данных ячейки
    table_data[row][column] = text

# Вывод полученной таблицы
df = pd.DataFrame(table_data)
# Сохранение DataFrame в формате CSV
df.to_csv('table.csv', index=False, header=False)

[((8, 8), (177, 37)), ((8, 37), (177, 65)), ((8, 65), (177, 178)), ((177, 8), (424, 37)), ((177, 37), (424, 65)), ((177, 65), (424, 178)), ((424, 8), (670, 37)), ((424, 37), (670, 65)), ((424, 65), (670, 178)), ((670, 8), (839, 37)), ((670, 37), (839, 65)), ((670, 65), (839, 178))]


In [212]:
import cv2
import numpy as np

# Загрузка изображения
image = file_array[0]

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применение порогового преобразования для бинаризации изображения
ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Применение морфологического преобразования для удаления шума и замыкания линий
kernel = np.ones((5, 5), np.uint8)
morph_image = cv2.morphologyEx(threshold_image, cv2.MORPH_CLOSE, kernel)

# Нахождение контуров на изображении
contours, hierarchy = cv2.findContours(morph_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Создание списка координат ограничивающих рамок ячеек
cell_boxes = []

# Фильтрация контуров и определение ограничивающих рамок ячеек
for contour in contours:
    # Определение площади контура
    area = cv2.contourArea(contour)

    # Фильтрация контуров по площади
    if area > 3000:
        # Определение ограничивающей рамки контура
        x, y, w, h = cv2.boundingRect(contour)

        # Добавление ограничивающей рамки в список
        cell_boxes.append((x, y, x + w, y + h))

# Вывод координат ограничивающих рамок ячеек
for box in cell_boxes:
    x1, y1, x2, y2 = box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Отображение изображения с выделенными ячейками
cv2.imshow('Image with Cells', image)
cv2.waitKey(0)
cv2.destroyAllWindows()